# Augmentation_7
Custom Ensemble: Created a different synthesized dataset for each tree in the ensemble, using slightly different parameters.

## Import libraries

In [31]:
#import libraries
import pandas as pd
import numpy as np
import random as rand

In [32]:
#import datasets
data = pd.read_csv("./dataset.csv")
pd.set_option('display.max_row',None)
pd.set_option('display.max_column',None)

## Separate Petting/Non-Petting

In [33]:
#separate the datasets Petting and the datasets that are not Petting
petting_d= data[data['Petting'] == 1]
non_petting_d= data[data['Petting'] == 0]

In [34]:
#Datasets preprocessing
def data_preprocessing(data):
    data.reset_index(inplace=True)
    data=data.drop(['index'],axis=1)
    data= data.drop(['Eating / Cooking', 'Moved It', 'Petting','TV / Radio','Talking'], axis=1)
    return data

## SMOTE

In [35]:
class SMOTE:
    def __init__(self,data,k,pad,synth_number,skew):
        self.data=data
        self.k=k
        self.pad=pad
        self.synth_n=synth_number
        self.skew = skew
        
    # Make the dataframe into list and put the index
    def making_list(self,data):
        data_list=data.values.tolist()
        a=0
        for x in data_list:
            x.insert(0,a)                                       #put the index of the data in the front
            a+=1
        return data_list

    # Calculate eucledian distance
    def eucledian(self,p1,p2):
        dist = np.sqrt(np.sum((p1-p2)**2))
        return dist
    
    # SMOTE algorithm
    def smote_al (self):
        data=self.making_list(self.data)
        feat_cnt = len(data[0])-1
        feat_start = 1                                          #start position of features in dataset, usually first column are IDs
        data_avg = [0 for i in range(feat_start+feat_cnt)] 
        data_sd = [0 for i in range(feat_start+feat_cnt)]
        data_med = [0 for i in range(feat_start+feat_cnt)]      #med = median
        data_skew = [0 for i in range(feat_start+feat_cnt)]
        k = self.k                                               #number of nearest neighbors (e.g. 3)
        pad = self.pad                                           #Padding - percentage of std dev to add to each feature (2 = 95th percentile)
        skew_x = self.skew                                       #Skew factor ... >1 increases skew, <1 decreases skew, =1 no effect
        synth_n = self.synth_n                                   #number of synthetic samples to make
        synth = [[] for i in range(synth_n)]                    #placeholder array for synthetic dataset
        nn= [[] for i in range(len(data))]
        
        #Find nearest neighbors
        dist_list=[]                              
        for i in data:                                      
            idx = i[0]                                          #get current sample index
            for m in range(len(data)):
                dist=self.eucledian(np.array(data[idx][1:]),np.array(data[m][1:])) #get the eucledian distance between the current data and other data
                a=(dist,idx,m)
                dist_list.append(list(a))
            dist_list.sort(key=lambda x:x[0])                  #sort the list by the distance
            for j in range(k):
                nn[idx].append(dist_list[j+1][2])               #append the data index
            dist_list=[]
        
        # Calc std devs 
        '''calculate the std dev for feature j across all rows of data, put in data_sd array'''
        for j in range(feat_start, feat_start+feat_cnt):
            temp_arr=[]
            for i in range(len(data)):
                temp_arr.append(data[i][j])                     #if dataset is numpy array, this could be done more cleanly w/o pulling data out to temp array
            data_avg[j] = np.mean(temp_arr)
            data_sd[j] = np.std(temp_arr)
            data_med[j] = np.median(temp_arr)
        
        #Calc skewness
        '''calculate skewness of dataset for each feature j'''
        for j in range(feat_start, feat_start+feat_cnt):
            if data_avg[j]-data_med[j]==0 :  # when avg, med, sd = 1.0, 1.0, 0.0
                data_skew[j] = 0
            else:
                data_skew[j] = (data_avg[j]-data_med[j])/ data_sd[j]
        
        # Create synthetic samples
        '''Randomly selecting samples, then use the difference between nearest neighbors and std dev to calcuate new feature values with perturbation'''
        for i in range(synth_n): 
            idx = rand.randint(0, len(data)-1)                      #get random sample index            
            for j in range(feat_start, feat_start+feat_cnt):
                diff = 0                                            #variable for holding difference between nearest neighbor
                gap = rand.uniform(-1.0,1.0)
                for q in range(k):
                    temp = nn[idx][q]
                    diff += abs(data[temp][j]) - abs(data[idx][j])      #sum absolute difference across all nearest neighbors
                diff = diff/k                                           #divide total by k to get avg diff
                diff = diff + (data_sd[j]*pad)                          #add in the std_dev * padding  to difference to create perturbations                
                gap2 = rand.uniform(0, 1.0)                             #random decimal number between -1 and 1
                skew_shift = (data_skew[j] * skew_x) - data_skew[j]     #calc skew_shift based on skew_x value
                skew_shift = data_avg[j] * skew_shift * gap2            #Apply skew_shift to average value, to get shift in unit_scale
                
                val = data[idx][j] + (diff*gap) + skew_shift
                synth[i].append(val)
                
        #Make into Dataframe
        '''Append synth array onto the original dataset array'''
        len_data=len(self.data)
        for i in range(self.synth_n):
            self.data.loc[len_data+i]=synth[i]
        return self.data

In [36]:
# Merging petting and non_petting datasets into one dataframe
def into_one_dataframe(k, pad, synth_size, skew, t_1, t_0):
    p=SMOTE(t_1, k, pad, synth_size+5, skew)
    q=SMOTE(t_0, k, pad, synth_size, skew)
    petting_data=p.smote_al()
    non_petting_data=q.smote_al()
    petting_data['Petting'] = 1
    non_petting_data['Petting'] = 0
    merged_data=pd.concat([petting_data,non_petting_data]) 
    return merged_data

## Create Custom Ensemble

In [37]:
# Import libraries
import numpy as np
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [38]:
def create_diff_synthData(dataset):    
    X_train = dataset.drop('Petting', axis=1)
    y_train = dataset['Petting']    
    return X_train, y_train

In [39]:
def create_ensemble(paramSet, cnt, t_1, t_0):
    # Create ensembles
    ensemble = []
    num_trees = len(paramSet)
    xtest = []
    ytest = []
    # Generate a different synthetic dataset for each tree in the ensemble
    while cnt>0:
        for i in range(num_trees):
            
            # Create synth dataset for each tree
            data_after_smote = into_one_dataframe(paramSet[i][0], paramSet[i][1], 
                                                  paramSet[i][2], paramSet[i][3], t_1.copy(), t_0.copy())
            X, y = create_diff_synthData(data_after_smote)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
            xtest.append(X_test)
            ytest.append(y_test)
            
            #Decision Tree
            tree = DecisionTreeClassifier()
            tree.fit(X_train, y_train)
            ensemble.append(tree)
        cnt-=1
    return ensemble, xtest, ytest

In [40]:
def evaluate(X_test_list, y_test_list, ensemble):
    # Evaluating ensemple
    acc = np.zeros((len(X_test_list), len(ensemble)))
    auc = np.zeros((len(X_test_list), len(ensemble)))    
    for i, tree in enumerate(ensemble):
        # Confusion Matirx
        y_pred = tree.predict(X_test_list[i])
        y_pred_prob = tree.predict_proba(X_test_list[i])[:, 1]
        acc[:, i] = accuracy_score(y_test_list[i], y_pred)
        auc[:, i] = roc_auc_score(y_test_list[i], y_pred_prob)

    #Evaluate
    accuracy = np.mean(acc)
    auc_mean = np.mean(auc)
    print("Accuracy: ", accuracy)
    print("AUC: ", auc_mean)

## Main Function

In [41]:
def main_7(paramSet, cnt, t_1, t_0):
    ensemble, X_test_list, y_test_list = create_ensemble(paramSet, cnt, t_1, t_0)
    evaluate(X_test_list, y_test_list, ensemble)

In [42]:
petting=data_preprocessing(petting_d)
non_petting=data_preprocessing(non_petting_d)

In [43]:
# Run one by one
# [k, pad, synth_size, skew]
params_list = [[2, 1, 40, 0.01], [2, 1, 40, 0.5], [2, 1, 40, 1.0], [2, 1, 40, 1.5], [2, 1, 40, 2.0],
              [2, 1, 80, 0.01], [2, 1, 80, 0.5], [2, 1, 80, 1.0], [2, 1, 80, 1.5], [2, 1, 80, 2.0],
              [2, 2, 40, 0.01], [2, 2, 40, 0.5], [2, 2, 40, 1.0], [2, 2, 40, 1.5], [2, 2, 40, 2.0],
              [2, 2, 80, 0.01], [2, 2, 80, 0.5], [2, 2, 80, 1.0], [2, 2, 80, 1.5], [2, 2, 80, 2.0],
              [3, 1, 40, 0.01], [3, 1, 40, 0.5], [3, 1, 40, 1.0], [3, 1, 40, 1.5], [3, 1, 40, 2.0],
              [3, 1, 80, 0.01], [3, 1, 80, 0.5], [3, 1, 80, 1.0], [3, 1, 80, 1.5], [3, 1, 80, 2.0],
              [3, 2, 40, 0.01], [3, 2, 40, 0.5], [3, 2, 40, 1.0], [3, 2, 40, 1.5], [3, 2, 40, 2.0],
              [3, 2, 80, 0.01], [3, 2, 80, 0.5], [3, 2, 80, 1.0], [3, 2, 80, 1.5], [3, 2, 80, 2.0],
              [4, 1, 40, 0.01], [4, 1, 40, 0.5], [4, 1, 40, 1.0], [4, 1, 40, 1.5], [4, 1, 40, 2.0],
              [4, 1, 80, 0.01], [4, 1, 80, 0.5], [4, 1, 80, 1.0], [4, 1, 80, 1.5], [4, 1, 80, 2.0],
              [4, 2, 40, 0.01], [4, 2, 40, 0.5], [4, 2, 40, 1.0], [4, 2, 40, 1.5], [4, 2, 40, 2.0],
              [4, 2, 80, 0.01], [4, 2, 80, 0.5], [4, 2, 80, 1.0], [4, 2, 80, 1.5], [4, 2, 80, 2.0]]
# Size duplication ## ex) 2 times or 3 times --> 120 trees / 180 trees
cnt = 3

## ACC, AUC Result

In [44]:
main_7(params_list, cnt, petting, non_petting)

Accuracy:  0.8856172839506172
AUC:  0.8989940604792239
